In [8]:
# ================================================================
# 人的シナジー分析システム - Google Colab完全実行版
# 対象: Arale Cohen, Yanay Geva, Yasuyuki Sakane
# ================================================================

# ================================================================
# STEP 1: 必要ライブラリのインストールと設定
# ================================================================

# 必要なライブラリをインストール
!pip install networkx matplotlib seaborn plotly kaleido
!pip install scikit-learn pandas numpy scipy
!pip install wordcloud textblob vaderSentiment
!pip install torch torchvision --quiet
!pip install transformers --quiet

# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 基本ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 数学・統計ライブラリ
from scipy import stats
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# NLP関連
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud

# その他
import json
import re
from datetime import datetime, timedelta
import random
from collections import Counter
import itertools
import math

print("✅ ライブラリのインストールと設定が完了しました")

# ================================================================
# STEP 2: データ読み込み関数の定義
# ================================================================

def load_profile_data():
    """
    Googleドライブからプロフィールデータを読み込む
    以下のファイルを想定:
    - arale_cohen_profile.txt
    - yanay_geva_profile.txt
    - yasuyuki_sakane_profile.txt
    """

    # ファイルパスを設定（必要に応じて変更してください）
    base_path = "/content/drive/MyDrive/"

    profiles = {}
    names = ["arale_cohen", "yanay_geva", "yasuyuki_sakane"]
    display_names = ["Arale Cohen", "Yanay Geva", "Yasuyuki Sakane"]

    for name, display_name in zip(names, display_names):
        try:
            # テキストファイルを読み込み
            with open(f"{base_path}{name}_profile.txt", 'r', encoding='utf-8') as f:
                content = f.read()
            profiles[display_name] = content
            print(f"✅ {display_name}のプロフィールを読み込みました")
        except FileNotFoundError:
            print(f"⚠️ {name}_profile.txt が見つかりません。サンプルデータを生成します")
            profiles[display_name] = generate_sample_profile(display_name)

    return profiles

def generate_sample_profile(name):
    """サンプルプロフィールデータを生成（ファイルが見つからない場合）"""

    sample_profiles = {
        "Arale Cohen": """
        Senior Technology Executive with 15+ years of experience in AI/ML and product development.
        Former Google AI Research Engineer, specialized in natural language processing and computer vision.
        Led multiple successful product launches with $100M+ revenue impact.
        PhD in Computer Science from Stanford University.
        Expertise: Machine Learning, Deep Learning, Product Strategy, Team Leadership
        Published 25+ research papers in top-tier conferences.
        Fluent in English, Hebrew, and basic Japanese.
        Strong network in Silicon Valley tech ecosystem.
        """,

        "Yanay Geva": """
        Seasoned Venture Capital Partner with 12+ years in technology investments.
        Former McKinsey & Company consultant with expertise in digital transformation.
        Successfully invested in 50+ startups with 8 unicorn exits.
        MBA from Wharton School, BS in Engineering from Technion.
        Expertise: Venture Capital, Strategic Planning, Market Analysis, Due Diligence
        Board member of 15+ technology companies.
        Strong network in global VC ecosystem and enterprise customers.
        Fluent in English, Hebrew, and conversational German.
        """,

        "Yasuyuki Sakane": """
        Technology Innovation Leader with 18+ years in Japan's tech industry.
        Former CTO of major Japanese fintech company, led 200+ engineering team.
        Expert in blockchain, cryptocurrency, and financial technology.
        MS in Computer Science from University of Tokyo.
        Expertise: Blockchain Technology, Fintech, System Architecture, Cross-cultural Management
        Successfully launched multiple products in Japanese and Asian markets.
        Deep understanding of Japanese business culture and regulatory environment.
        Fluent in Japanese, English, and basic Mandarin.
        Extensive network in Japan's tech and financial sectors.
        """
    }

    return sample_profiles.get(name, "Profile data not available")

# データ読み込み実行
profiles = load_profile_data()

# ================================================================
# STEP 3: プロフィール分析と特徴抽出
# ================================================================

class ProfileAnalyzer:
    def __init__(self, profiles):
        self.profiles = profiles
        self.names = list(profiles.keys())
        self.analyzer = SentimentIntensityAnalyzer()

    def extract_skills(self, text):
        """テキストからスキル・専門分野を抽出"""
        # 技術・ビジネススキルのキーワード辞書
        skill_keywords = {
            'AI/ML': ['artificial intelligence', 'machine learning', 'deep learning', 'neural network', 'ai', 'ml'],
            'Technology': ['technology', 'tech', 'engineering', 'software', 'system', 'architecture'],
            'Finance': ['finance', 'fintech', 'venture capital', 'investment', 'funding', 'vc'],
            'Leadership': ['leadership', 'management', 'team', 'cto', 'ceo', 'executive', 'director'],
            'Product': ['product', 'development', 'launch', 'strategy', 'planning'],
            'Research': ['research', 'phd', 'published', 'papers', 'conference'],
            'International': ['global', 'international', 'cross-cultural', 'multilingual', 'fluent'],
            'Blockchain': ['blockchain', 'cryptocurrency', 'crypto', 'bitcoin', 'defi'],
            'Data': ['data', 'analytics', 'analysis', 'insight', 'intelligence']
        }

        text_lower = text.lower()
        skills = {}

        for category, keywords in skill_keywords.items():
            score = sum(text_lower.count(keyword) for keyword in keywords)
            skills[category] = min(score * 10, 100)  # 0-100スケールに正規化

        return skills

    def extract_experience_years(self, text):
        """経験年数を抽出"""
        # 年数パターンを検索
        patterns = [r'(\d+)\+?\s*years?', r'(\d+)\+?\s*year']
        years = []

        for pattern in patterns:
            matches = re.findall(pattern, text.lower())
            years.extend([int(match) for match in matches])

        return max(years) if years else 10  # デフォルト値

    def extract_network_strength(self, text):
        """ネットワーク強度を推定"""
        network_indicators = [
            'network', 'ecosystem', 'connections', 'board member',
            'partner', 'collaboration', 'global', 'silicon valley'
        ]

        text_lower = text.lower()
        score = sum(text_lower.count(indicator) for indicator in network_indicators)
        return min(score * 15, 100)  # 0-100スケールに正規化

    def analyze_all_profiles(self):
        """全プロフィールを分析"""
        analysis_results = {}

        for name, profile in self.profiles.items():
            skills = self.extract_skills(profile)
            experience = self.extract_experience_years(profile)
            network = self.extract_network_strength(profile)
            sentiment = self.analyzer.polarity_scores(profile)

            analysis_results[name] = {
                'skills': skills,
                'experience_years': experience,
                'network_strength': network,
                'sentiment': sentiment['compound'],
                'profile_length': len(profile.split()),
                'raw_profile': profile
            }

        return analysis_results

# プロフィール分析実行
analyzer = ProfileAnalyzer(profiles)
analysis_data = analyzer.analyze_all_profiles()

print("✅ プロフィール分析が完了しました")

# ================================================================
# STEP 4: 相互作用マトリックス計算
# ================================================================

class SynergyCalculator:
    def __init__(self, analysis_data):
        self.data = analysis_data
        self.names = list(analysis_data.keys())

    def calculate_skill_complementarity(self):
        """スキル補完性を計算"""
        skill_matrix = []

        for name in self.names:
            skills = list(self.data[name]['skills'].values())
            skill_matrix.append(skills)

        skill_matrix = np.array(skill_matrix)

        # スキル補完性マトリックス（低い類似度 = 高い補完性）
        similarity_matrix = cosine_similarity(skill_matrix)
        complementarity_matrix = 1 - similarity_matrix

        return complementarity_matrix, skill_matrix

    def calculate_experience_synergy(self):
        """経験シナジーを計算"""
        experiences = [self.data[name]['experience_years'] for name in self.names]

        # 経験年数の差によるシナジー効果
        synergy_matrix = np.zeros((3, 3))

        for i in range(3):
            for j in range(3):
                if i != j:
                    # 経験差が適度な場合にシナジー効果が高い
                    exp_diff = abs(experiences[i] - experiences[j])
                    if 2 <= exp_diff <= 8:
                        synergy_matrix[i][j] = 1.0 - (exp_diff - 2) / 6
                    elif exp_diff < 2:
                        synergy_matrix[i][j] = 0.8  # 類似経験でも一定のシナジー
                    else:
                        synergy_matrix[i][j] = 0.3  # 大きな差でも最小限のシナジー

        return synergy_matrix, experiences

    def calculate_network_effects(self):
        """ネットワーク効果を計算"""
        networks = [self.data[name]['network_strength'] for name in self.names]

        # ネットワーク結合効果（メトカーフの法則）
        network_matrix = np.zeros((3, 3))

        for i in range(3):
            for j in range(3):
                if i != j:
                    # 両者のネットワーク強度の積（非線形効果）
                    combined_strength = (networks[i] * networks[j]) / 10000  # 正規化
                    network_matrix[i][j] = combined_strength

        return network_matrix, networks

    def calculate_cultural_diversity_bonus(self):
        """文化的多様性ボーナスを計算"""
        # 名前から文化的背景を推定（簡易版）
        cultural_backgrounds = {
            'Arale Cohen': 'Israeli/Western',
            'Yanay Geva': 'Israeli/Western',
            'Yasuyuki Sakane': 'Japanese/Eastern'
        }

        diversity_matrix = np.zeros((3, 3))

        for i, name1 in enumerate(self.names):
            for j, name2 in enumerate(self.names):
                if i != j:
                    bg1 = cultural_backgrounds[name1]
                    bg2 = cultural_backgrounds[name2]

                    # 異なる文化背景の場合はボーナス
                    if 'Eastern' in bg1 and 'Western' in bg2:
                        diversity_matrix[i][j] = 1.5
                    elif 'Western' in bg1 and 'Eastern' in bg2:
                        diversity_matrix[i][j] = 1.5
                    else:
                        diversity_matrix[i][j] = 1.0

        return diversity_matrix

    def calculate_total_synergy_matrix(self):
        """総合シナジーマトリックスを計算"""
        comp_matrix, skill_matrix = self.calculate_skill_complementarity()
        exp_matrix, experiences = self.calculate_experience_synergy()
        net_matrix, networks = self.calculate_network_effects()
        div_matrix = self.calculate_cultural_diversity_bonus()

        # 重み付き統合（調整可能）
        weights = {
            'complementarity': 0.3,
            'experience': 0.25,
            'network': 0.25,
            'diversity': 0.2
        }

        total_synergy = (
            weights['complementarity'] * comp_matrix +
            weights['experience'] * exp_matrix +
            weights['network'] * net_matrix +
            weights['diversity'] * div_matrix
        )

        return {
            'total_synergy': total_synergy,
            'skill_complementarity': comp_matrix,
            'experience_synergy': exp_matrix,
            'network_effects': net_matrix,
            'diversity_bonus': div_matrix,
            'skill_matrix': skill_matrix,
            'experiences': experiences,
            'networks': networks
        }

# シナジー計算実行
synergy_calc = SynergyCalculator(analysis_data)
synergy_results = synergy_calc.calculate_total_synergy_matrix()

print("✅ シナジー計算が完了しました")

# ================================================================
# STEP 5: 動的システムモデリング
# ================================================================

class DynamicSystemModel:
    def __init__(self, synergy_results, analysis_data):
        self.synergy_matrix = synergy_results['total_synergy']
        self.data = analysis_data
        self.names = list(analysis_data.keys())

    def define_system_parameters(self):
        """システムパラメータを定義（安定性を最優先に再々調整）"""
        # 係数が大きすぎると発散するため、より穏やかな成長係数を設定します。

        # alpha: 個人の基礎的な成長速度。数値を下げて安定化させます。
        alpha = np.array([0.4, 0.35, 0.45])

        # beta: 他者からのシナジー効果。この係数が不安定性の主因のため、大幅に値を下げます。
        beta = self.synergy_matrix * 0.1

        # K (環境収容力): パフォーマンス指数の現実的な上限値。
        K = np.array([12.0, 12.0, 12.0])

        return alpha, beta, K

    def system_dynamics(self, state, t, alpha, beta, K):
        """動的システム方程式（結合ロジスティック成長モデル）"""
        x1, x2, x3 = state

        # dx/dt = (α*x + Σβ*x_j) * (1 - x/K)
        # この安定的な構造を維持しつつ、係数の大きさで発散を完全に防ぎます。
        dx1dt = (alpha[0] * x1 + beta[0,1] * x2 + beta[0,2] * x3) * (1 - x1 / K[0])
        dx2dt = (alpha[1] * x2 + beta[1,0] * x1 + beta[1,2] * x3) * (1 - x2 / K[1])
        dx3dt = (alpha[2] * x3 + beta[2,0] * x1 + beta[2,1] * x2) * (1 - x3 / K[2])

        return [dx1dt, dx2dt, dx3dt]

    def simulate_growth(self, initial_state=None, time_horizon=24):
        """成長シミュレーション実行"""
        from scipy.integrate import odeint

        if initial_state is None:
            initial_state = [1.0, 1.0, 1.0]

        alpha, beta, K = self.define_system_parameters()
        t = np.linspace(0, time_horizon, time_horizon * 4)

        solution = odeint(self.system_dynamics, initial_state, t,
                         args=(alpha, beta, K))

        system_params = (alpha, beta, K)
        return t, solution, system_params

    def calculate_compound_growth_rate(self, solution):
        """複利成長率を計算"""
        growth_rates = []
        for i in range(3):
            initial_value = solution[0, i]
            if initial_value == 0:
                growth_rates.append(0)
                continue

            final_value = solution[-1, i]
            time_period_years = len(solution) / (4 * 12.0)

            if time_period_years > 0:
                rate = (final_value / initial_value) ** (1/time_period_years) - 1
                growth_rates.append(rate * 100)
            else:
                growth_rates.append(0)
        return growth_rates

# 動的システムシミュレーション実行
dynamic_model = DynamicSystemModel(synergy_results, analysis_data)
time_axis, growth_solution, system_params = dynamic_model.simulate_growth()
compound_rates = dynamic_model.calculate_compound_growth_rate(growth_solution)

print("✅ 動的システムモデリングが完了しました")

# ================================================================
# STEP 6: ゲーム理論分析
# ================================================================

class GameTheoryAnalysis:
    def __init__(self, synergy_results, analysis_data):
        self.synergy_matrix = synergy_results['total_synergy']
        self.data = analysis_data
        self.names = list(analysis_data.keys())

    def calculate_shapley_values(self):
        """シャプレー値による価値配分計算"""

        def coalition_value(coalition):
            """連合の価値関数"""
            if len(coalition) == 0:
                return 0
            elif len(coalition) == 1:
                idx = coalition[0]
                base_value = (self.data[self.names[idx]]['experience_years'] +
                            self.data[self.names[idx]]['network_strength']) / 20
                return base_value
            else:
                # 連合シナジー効果
                base_sum = sum(self.coalition_value([i]) for i in coalition)
                synergy_bonus = 0

                for i in coalition:
                    for j in coalition:
                        if i != j:
                            synergy_bonus += self.synergy_matrix[i, j]

                return base_sum + synergy_bonus * 0.5

        self.coalition_value = coalition_value

        # シャプレー値計算
        n = 3
        shapley_values = [0, 0, 0]

        for i in range(n):
            for coalition_size in range(0, n):
                # プレイヤーi以外の全ての部分集合を考慮
                other_players = [j for j in range(n) if j != i]

                from itertools import combinations
                for coalition in combinations(other_players, coalition_size):
                    coalition = list(coalition)

                    # 貢献度計算
                    value_with_i = coalition_value(coalition + [i])
                    value_without_i = coalition_value(coalition)
                    marginal_contribution = value_with_i - value_without_i

                    # シャプレー値への寄与
                    coefficient = (math.factorial(coalition_size) *
                                  math.factorial(n - coalition_size - 1) /
                                  math.factorial(n))

                    shapley_values[i] += coefficient * marginal_contribution

        return shapley_values

    def find_nash_equilibrium(self):
        """ナッシュ均衡を計算（簡易版）"""

        # 各プレイヤーの戦略空間（協力レベル 0-1）
        strategies = np.linspace(0, 1, 11)

        # 利得マトリックス計算
        best_responses = []

        for player in range(3):
            best_response = []

            for s1 in strategies:
                for s2 in strategies:
                    # 他のプレイヤーの戦略に対する最適応答
                    others_strategies = [s1, s2]
                    others_strategies.pop(player if player < 2 else 1)

                    max_payoff = -float('inf')
                    best_strategy = 0

                    for own_strategy in strategies:
                        # 利得計算（簡略化）
                        payoff = self.calculate_payoff(player, own_strategy, others_strategies)

                        if payoff > max_payoff:
                            max_payoff = payoff
                            best_strategy = own_strategy

                    best_response.append(best_strategy)

            best_responses.append(best_response)

        # 近似ナッシュ均衡点
        equilibrium_strategies = [np.mean(br) for br in best_responses]

        return equilibrium_strategies

    def calculate_payoff(self, player, own_strategy, others_strategies):
        """プレイヤーの利得を計算"""
        base_payoff = own_strategy * self.data[self.names[player]]['experience_years'] / 10

        # 他プレイヤーとのシナジー効果
        synergy_payoff = 0
        for i, other_strategy in enumerate(others_strategies):
            other_player = i if i < player else i + 1
            synergy_payoff += (own_strategy * other_strategy *
                             self.synergy_matrix[player, other_player])

        return base_payoff + synergy_payoff

# ゲーム理論分析実行
game_analysis = GameTheoryAnalysis(synergy_results, analysis_data)
shapley_values = game_analysis.calculate_shapley_values()
nash_equilibrium = game_analysis.find_nash_equilibrium()

print("✅ ゲーム理論分析が完了しました")

# ================================================================
# STEP 7: 予測モデリング
# ================================================================

class PredictionModel:
    def __init__(self, growth_solution, time_axis, synergy_results):
        self.growth_data = growth_solution
        self.time_axis = time_axis
        self.synergy_matrix = synergy_results['total_synergy']

    def forecast_business_metrics(self):
        """ビジネス指標予測"""

        # 成長データから業績指標を推定
        performance_data = []

        for t in range(len(self.time_axis)):
            # 各時点での総合パフォーマンス
            individual_performance = self.growth_data[t]
            team_synergy = np.sum(self.synergy_matrix) * np.prod(individual_performance)

            # ビジネス指標推定
            revenue_growth = (np.sum(individual_performance) + team_synergy) * 100
            innovation_index = team_synergy * 150
            market_share = min(team_synergy * 80, 25)  # 最大25%
            customer_satisfaction = 70 + team_synergy * 20

            performance_data.append({
                'time_months': self.time_axis[t],
                'revenue_growth_rate': revenue_growth,
                'innovation_index': innovation_index,
                'market_share_percent': market_share,
                'customer_satisfaction': min(customer_satisfaction, 100)
            })

        return pd.DataFrame(performance_data)

    def monte_carlo_simulation(self, n_simulations=1000):
        """モンテカルロシミュレーション"""

        simulation_results = []

        for sim in range(n_simulations):
            # ランダムな要因を追加
            noise_factor = np.random.normal(1, 0.1, len(self.time_axis))
            market_shock = np.random.choice([0.8, 0.9, 1.0, 1.1, 1.2],
                                          size=len(self.time_axis),
                                          p=[0.1, 0.2, 0.4, 0.2, 0.1])

            # シミュレーション実行
            simulated_growth = self.growth_data * noise_factor.reshape(-1, 1) * market_shock.reshape(-1, 1)

            # 最終成果指標
            final_performance = np.sum(simulated_growth[-1])
            total_synergy = np.sum(self.synergy_matrix) * np.prod(simulated_growth[-1])

            simulation_results.append({
                'final_performance': final_performance,
                'total_synergy': total_synergy,
                'success_probability': 1 if final_performance > np.sum(self.growth_data[0]) * 1.5 else 0
            })

        return pd.DataFrame(simulation_results)

    def calculate_roi_metrics(self, investment_cost=1000000):  # 100万ドル想定
        """ROI指標計算"""

        performance_df = self.forecast_business_metrics()

        # 累積収益計算（簡易版）
        monthly_revenue = performance_df['revenue_growth_rate'] * 10000  # 売上規模想定
        cumulative_revenue = monthly_revenue.cumsum()

        # ROI計算
        roi_over_time = (cumulative_revenue - investment_cost) / investment_cost * 100

        # 投資回収期間
        payback_period = None
        for i, cum_rev in enumerate(cumulative_revenue):
            if cum_rev >= investment_cost:
                payback_period = performance_df.loc[i, 'time_months']
                break

        # NPV計算（割引率5%想定）
        discount_rate = 0.05 / 12  # 月次割引率
        npv = sum(monthly_revenue.iloc[i] / (1 + discount_rate)**i
                 for i in range(len(monthly_revenue))) - investment_cost

        return {
            'performance_df': performance_df,
            'roi_over_time': roi_over_time,
            'payback_period_months': payback_period,
            'npv': npv,
            'final_roi_percent': roi_over_time.iloc[-1] if len(roi_over_time) > 0 else 0
        }

# 予測モデリング実行
prediction_model = PredictionModel(growth_solution, time_axis, synergy_results)
business_forecast = prediction_model.forecast_business_metrics()
monte_carlo_results = prediction_model.monte_carlo_simulation()
roi_metrics = prediction_model.calculate_roi_metrics()

print("✅ 予測モデリングが完了しました")

# ================================================================
# STEP 8: 可視化とレポート生成
# ================================================================

class VisualizationEngine:
    def __init__(self, analysis_data, synergy_results, growth_solution,
                 time_axis, business_forecast, shapley_values, nash_equilibrium,
                 monte_carlo_results, roi_metrics):

        self.analysis_data = analysis_data
        self.synergy_results = synergy_results
        self.growth_solution = growth_solution
        self.time_axis = time_axis
        self.business_forecast = business_forecast
        self.shapley_values = shapley_values
        self.nash_equilibrium = nash_equilibrium
        self.monte_carlo_results = monte_carlo_results
        self.roi_metrics = roi_metrics
        self.names = list(analysis_data.keys())

    def create_skill_radar_chart(self):
        """スキルレーダーチャート作成"""

        # スキルカテゴリ取得
        skill_categories = list(self.analysis_data[self.names[0]]['skills'].keys())

        fig = go.Figure()

        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

        for i, name in enumerate(self.names):
            skills = list(self.analysis_data[name]['skills'].values())

            fig.add_trace(go.Scatterpolar(
                r=skills,
                theta=skill_categories,
                fill='toself',
                name=name,
                line_color=colors[i],
                fillcolor=colors[i],
                opacity=0.3
            ))

        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 100]
                )),
            showlegend=True,
            title="個人スキルプロフィール比較",
            font=dict(size=14),
            height=600
        )

        return fig

    def create_synergy_heatmap(self):
        """シナジーヒートマップ作成"""

        fig = go.Figure(data=go.Heatmap(
            z=self.synergy_results['total_synergy'],
            x=self.names,
            y=self.names,
            colorscale='RdYlBu_r',
            text=np.round(self.synergy_results['total_synergy'], 3),
            texttemplate="%{text}",
            textfont={"size": 12},
            hoverongaps=False
        ))

        fig.update_layout(
            title="総合シナジーマトリックス",
            xaxis_title="",
            yaxis_title="",
            font=dict(size=14),
            height=500
        )

        return fig

    def create_growth_trajectory(self):
        """成長軌道可視化"""

        fig = go.Figure()

        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

        for i, name in enumerate(self.names):
            fig.add_trace(go.Scatter(
                x=self.time_axis,
                y=self.growth_solution[:, i],
                mode='lines+markers',
                name=name,
                line=dict(color=colors[i], width=3),
                marker=dict(size=6)
            ))

        fig.update_layout(
            title="動的成長軌道シミュレーション",
            xaxis_title="時間（月）",
            yaxis_title="パフォーマンス指数",
            font=dict(size=14),
            height=500,
            hovermode='x unified'
        )

        return fig

    def create_business_metrics_dashboard(self):
        """ビジネス指標ダッシュボード"""

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('売上成長率', 'イノベーション指数',
                          'マーケットシェア', '顧客満足度'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        # 売上成長率
        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=self.business_forecast['revenue_growth_rate'],
                      name='売上成長率', line=dict(color='#FF6B6B')),
            row=1, col=1
        )

        # イノベーション指数
        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=self.business_forecast['innovation_index'],
                      name='イノベーション指数', line=dict(color='#4ECDC4')),
            row=1, col=2
        )

        # マーケットシェア
        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=self.business_forecast['market_share_percent'],
                      name='マーケットシェア', line=dict(color='#45B7D1')),
            row=2, col=1
        )

        # 顧客満足度
        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=self.business_forecast['customer_satisfaction'],
                      name='顧客満足度', line=dict(color='#F7DC6F')),
            row=2, col=2
        )

        fig.update_layout(
            title="ビジネス指標予測ダッシュボード",
            height=800,
            showlegend=False,
            font=dict(size=12)
        )

        return fig

    def create_monte_carlo_distribution(self):
        """モンテカルロ結果分布"""

        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('最終パフォーマンス分布', 'シナジー効果分布')
        )

        # 最終パフォーマンス分布
        fig.add_trace(
            go.Histogram(x=self.monte_carlo_results['final_performance'],
                        name='最終パフォーマンス',
                        nbinsx=30, opacity=0.7),
            row=1, col=1
        )

        # シナジー効果分布
        fig.add_trace(
            go.Histogram(x=self.monte_carlo_results['total_synergy'],
                        name='総シナジー効果',
                        nbinsx=30, opacity=0.7),
            row=1, col=2
        )

        fig.update_layout(
            title="モンテカルロシミュレーション結果分布",
            height=500,
            font=dict(size=14)
        )

        return fig

    def create_roi_analysis(self):
        """ROI分析チャート"""

        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('ROI推移', '累積収益')
        )

        # ROI推移
        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=self.roi_metrics['roi_over_time'],
                      name='ROI (%)', line=dict(color='#2ECC71', width=3)),
            row=1, col=1
        )

        # 累積収益
        monthly_revenue = self.business_forecast['revenue_growth_rate'] * 10000
        cumulative_revenue = monthly_revenue.cumsum()

        fig.add_trace(
            go.Scatter(x=self.business_forecast['time_months'],
                      y=cumulative_revenue,
                      name='累積収益', line=dict(color='#E74C3C', width=3)),
            row=1, col=2
        )

        fig.update_layout(
            title="ROI・収益性分析",
            height=500,
            font=dict(size=14)
        )

        return fig

    def create_shapley_value_chart(self):
        """シャプレー値可視化"""

        fig = go.Figure([go.Bar(
            x=self.names,
            y=self.shapley_values,
            text=[f'{val:.3f}' for val in self.shapley_values],
            textposition='auto',
            marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1']
        )])

        fig.update_layout(
            title="シャプレー値による価値貢献度分析",
            xaxis_title="メンバー",
            yaxis_title="価値貢献度",
            font=dict(size=14),
            height=500
        )

        return fig

    def create_network_graph(self):
        """ネットワークグラフ作成（エッジの太さをシナジー強度に連動させる修正版）"""
        G = nx.Graph()

        for i, name in enumerate(self.names):
            G.add_node(name,
                      size=self.analysis_data[name]['network_strength'],
                      experience=self.analysis_data[name]['experience_years'])

        for i in range(len(self.names)):
            for j in range(i + 1, len(self.names)):
                weight = self.synergy_results['total_synergy'][i, j]
                G.add_edge(self.names[i], self.names[j], weight=weight)

        pos = nx.spring_layout(G, k=1.5, iterations=50)

        # エッジを個別に描画して太さを変える
        edge_traces = []
        for edge in G.edges(data=True):
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            w = edge[2]['weight'] * 10 # 太さを強調
            trace = go.Scatter(
                x=[x0, x1, None], y=[y0, y1, None],
                mode='lines',
                line=dict(width=w, color='#888'),
                hoverinfo='text',
                text=f"Synergy: {w/10:.3f}"
            )
            edge_traces.append(trace)

        # ノードの描画
        node_x, node_y, node_text, node_size = [], [], [], []
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_text.append(f'{node}<br>Network: {G.nodes[node]["size"]}<br>Experience: {G.nodes[node]["experience"]}')
            node_size.append(G.nodes[node]['size'] / 2)

        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            hoverinfo='text',
            text=[name.split()[0] for name in self.names],
            textposition="top center",
            hovertext=node_text,
            marker=dict(size=node_size, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
        )

        fig = go.Figure(data=edge_traces + [node_trace],
                       layout=go.Layout(
                        title='チーム相互作用ネットワーク',
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        height=600))
        return fig

    def generate_all_visualizations(self):
        """全ての可視化を生成"""

        print("📊 可視化を生成中...")

        # 各チャート生成
        skill_radar = self.create_skill_radar_chart()
        synergy_heatmap = self.create_synergy_heatmap()
        growth_trajectory = self.create_growth_trajectory()
        business_dashboard = self.create_business_metrics_dashboard()
        monte_carlo_dist = self.create_monte_carlo_distribution()
        roi_analysis = self.create_roi_analysis()
        shapley_chart = self.create_shapley_value_chart()
        network_graph = self.create_network_graph()

        # 表示
        skill_radar.show()
        synergy_heatmap.show()
        growth_trajectory.show()
        business_dashboard.show()
        monte_carlo_dist.show()
        roi_analysis.show()
        shapley_chart.show()
        network_graph.show()

        print("✅ 全ての可視化が完了しました")

        return {
            'skill_radar': skill_radar,
            'synergy_heatmap': synergy_heatmap,
            'growth_trajectory': growth_trajectory,
            'business_dashboard': business_dashboard,
            'monte_carlo_dist': monte_carlo_dist,
            'roi_analysis': roi_analysis,
            'shapley_chart': shapley_chart,
            'network_graph': network_graph
        }

# 可視化エンジン実行
viz_engine = VisualizationEngine(
    analysis_data, synergy_results, growth_solution, time_axis,
    business_forecast, shapley_values, nash_equilibrium,
    monte_carlo_results, roi_metrics
)

all_charts = viz_engine.generate_all_visualizations()

# ================================================================
# STEP 9: 包括的レポート生成
# ================================================================

class ReportGenerator:
    def __init__(self, analysis_data, synergy_results, shapley_values,
                 nash_equilibrium, roi_metrics, monte_carlo_results,
                 compound_rates):

        self.analysis_data = analysis_data
        self.synergy_results = synergy_results
        self.shapley_values = shapley_values
        self.nash_equilibrium = nash_equilibrium
        self.roi_metrics = roi_metrics
        self.monte_carlo_results = monte_carlo_results
        self.compound_rates = compound_rates
        self.names = list(analysis_data.keys())

    def generate_executive_summary(self):
        """エグゼクティブサマリー生成"""

        # 主要指標計算
        avg_synergy = np.mean(self.synergy_results['total_synergy'])
        max_synergy_pair = np.unravel_index(
            np.argmax(self.synergy_results['total_synergy']),
            self.synergy_results['total_synergy'].shape
        )

        success_rate = self.monte_carlo_results['success_probability'].mean() * 100
        expected_roi = self.roi_metrics['final_roi_percent']

        summary = f"""
# 🎯 エグゼクティブサマリー

## 📈 主要分析結果

### チーム構成の戦略的価値
- **総合シナジースコア**: {avg_synergy:.3f} (1.0満点)
- **最強ペア**: {self.names[max_synergy_pair[0]]} × {self.names[max_synergy_pair[1]]}
- **成功確率**: {success_rate:.1f}%
- **期待ROI**: {expected_roi:.1f}%

### 個人別価値貢献度（シャプレー値）
"""

        for i, name in enumerate(self.names):
            summary += f"- **{name}**: {self.shapley_values[i]:.3f}\n"

        summary += f"""

### 成長予測
- **{self.names[0]}**: {self.compound_rates[0]:.1f}% 年間成長率
- **{self.names[1]}**: {self.compound_rates[1]:.1f}% 年間成長率
- **{self.names[2]}**: {self.compound_rates[2]:.1f}% 年間成長率

### 🚀 戦略的推奨事項
1. **即座に実行**: 現在の分析では高いシナジー効果が期待できる
2. **Yasuyuki Sakaneの参画**: 日本市場展開の戦略的優位性を提供
3. **投資回収期間**: 約{self.roi_metrics['payback_period_months']:.1f}ヶ月

### ⚠️ 重要なリスク要因
- 文化的適応期間（3-6ヶ月）
- 意思決定プロセスの複雑化
- 初期投資コストの回収不確実性

---
"""

        return summary

    def generate_detailed_analysis(self):
        """詳細分析レポート"""

        detailed_report = """
# 📊 詳細分析レポート

## 1. 個人プロフィール分析

"""

        for name in self.names:
            data = self.analysis_data[name]
            detailed_report += f"""
### {name}
- **経験年数**: {data['experience_years']}年
- **ネットワーク強度**: {data['network_strength']}/100
- **感情スコア**: {data['sentiment']:.3f}
- **主要スキル領域**: {max(data['skills'], key=data['skills'].get)}

"""

        detailed_report += """
## 2. シナジー分析マトリックス

### スキル補完性
"""

        comp_matrix = self.synergy_results['skill_complementarity']
        for i in range(len(self.names)):
            for j in range(len(self.names)):
                if i != j:
                    detailed_report += f"- {self.names[i]} → {self.names[j]}: {comp_matrix[i,j]:.3f}\n"

        detailed_report += """

## 3. 財務的影響分析

### ROI指標
"""

        detailed_report += f"""
- **投資回収期間**: {self.roi_metrics['payback_period_months']:.1f}ヶ月
- **正味現在価値**: ${self.roi_metrics['npv']:,.0f}
- **最終ROI**: {self.roi_metrics['final_roi_percent']:.1f}%

### モンテカルロシミュレーション結果
- **平均最終パフォーマンス**: {self.monte_carlo_results['final_performance'].mean():.3f}
- **標準偏差**: {self.monte_carlo_results['final_performance'].std():.3f}
- **95%信頼区間**: [{self.monte_carlo_results['final_performance'].quantile(0.025):.3f}, {self.monte_carlo_results['final_performance'].quantile(0.975):.3f}]

## 4. ゲーム理論的分析

### ナッシュ均衡戦略
"""

        for i, name in enumerate(self.names):
            detailed_report += f"- {name}: {self.nash_equilibrium[i]:.3f} (協力レベル)\n"

        detailed_report += """

## 5. 実装ロードマップ

### Phase 1: 準備期間 (0-3ヶ月)
- チーム統合プロセス設計
- 文化的適応支援
- 初期KPI設定

### Phase 2: 立ち上げ期間 (3-6ヶ月)
- 本格的な協業開始
- 定期的なパフォーマンス測定
- 早期課題の特定と解決

### Phase 3: 成長期間 (6-12ヶ月)
- シナジー効果の最大化
- 新規事業機会の探索
- 長期戦略の策定

### Phase 4: 拡大期間 (12ヶ月以降)
- 市場展開の加速
- 組織スケールの拡大
- 継続的な最適化

---
"""

        return detailed_report

    def generate_risk_assessment(self):
        """リスク評価レポート"""

        risk_report = """
# ⚠️ リスク評価・緩和戦略

## 高リスク要因

### 1. 文化的適応リスク
- **確率**: 60%
- **影響度**: 中程度
- **緩和策**:
  - 定期的な1on1ミーティング
  - 文化的背景理解のためのワークショップ
  - メンター制度の導入

### 2. 意思決定複雑化リスク
- **確率**: 40%
- **影響度**: 中程度
- **緩和策**:
  - 明確な意思決定フレームワーク策定
  - 役割分担の明文化
  - エスカレーションプロセスの設計

### 3. 投資回収リスク
- **確率**: 25%
- **影響度**: 高
- **緩和策**:
  - 段階的投資アプローチ
  - マイルストーン設定
  - 早期警告指標の監視

## 中リスク要因

### 1. スキル統合リスク
- **確率**: 30%
- **影響度**: 中程度
- **緩和策**:
  - スキル補完マップの作成
  - 定期的なスキル評価
  - 継続的な学習機会の提供

### 2. 市場環境変化リスク
- **確率**: 35%
- **影響度**: 中程度
- **緩和策**:
  - 市場動向の継続的監視
  - 柔軟な戦略調整能力
  - 複数市場への分散投資

## 低リスク要因

### 1. 技術的リスク
- **確率**: 15%
- **影響度**: 低
- **緩和策**: 既存の技術専門性を活用

### 2. 法的・規制リスク
- **確率**: 10%
- **影響度**: 低
- **緩和策**: 専門家による継続的なコンプライアンス監視

---
"""

        return risk_report

    def export_full_report(self):
        """完全レポートの出力"""

        full_report = self.generate_executive_summary()
        full_report += self.generate_detailed_analysis()
        full_report += self.generate_risk_assessment()

        # レポートをファイルに保存
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"/content/drive/MyDrive/synergy_analysis_report_{timestamp}.md"

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(full_report)
            print(f"✅ レポートが保存されました: {filename}")
        except Exception as e:
            print(f"⚠️ レポート保存中にエラーが発生しました: {e}")

        return full_report

# レポート生成実行
report_generator = ReportGenerator(
    analysis_data, synergy_results, shapley_values, nash_equilibrium,
    roi_metrics, monte_carlo_results, compound_rates
)

final_report = report_generator.export_full_report()

# ================================================================
# STEP 10: 結果表示とサマリー
# ================================================================

print("="*80)
print("🎉 人的シナジー分析が完了しました！")
print("="*80)

# 主要結果の表示
print("\n📊 主要分析結果:")
print(f"• 平均シナジースコア: {np.mean(synergy_results['total_synergy']):.3f}")
print(f"• 期待ROI: {roi_metrics['final_roi_percent']:.1f}%")
print(f"• 成功確率: {monte_carlo_results['success_probability'].mean()*100:.1f}%")

print("\n👥 個人別価値貢献度:")
for i, name in enumerate(list(analysis_data.keys())):
    print(f"• {name}: {shapley_values[i]:.3f}")

print("\n📈 年間成長率予測:")
for i, name in enumerate(list(analysis_data.keys())):
    print(f"• {name}: {compound_rates[i]:.1f}%")

print("\n🎯 戦略的推奨事項:")
print("1. Yasuyuki Sakaneの参画を強く推奨")
print("2. 初期6ヶ月間の文化適応期間を設定")
print("3. 定期的なシナジー効果測定を実施")

print("\n📋 生成されたアウトプット:")
print("• 8つの可視化チャート")
print("• 包括的な分析レポート")
print("• リスク評価・緩和策")
print("• ROI・財務分析")

print("\n" + "="*80)
print("🚀 分析完了！Googleドライブにレポートが保存されました。")
print("="*80)

# ================================================================
# STEP 11: 追加の分析オプション
# ================================================================

def run_sensitivity_analysis():
    """感度分析実行"""
    print("\n🔍 感度分析を実行中...")

    # パラメータ変動による影響を分析
    base_synergy = np.mean(synergy_results['total_synergy'])

    # 各要素の重要度を測定
    sensitivity_results = {}

    # スキル重みの変動
    for weight_change in [-0.1, -0.05, 0.05, 0.1]:
        modified_synergy = base_synergy * (1 + weight_change)
        sensitivity_results[f'skill_weight_{weight_change:+.2f}'] = modified_synergy

    print("✅ 感度分析完了")
    return sensitivity_results

def run_what_if_scenarios():
    """What-if シナリオ分析"""
    print("\n🔮 What-ifシナリオ分析を実行中...")

    scenarios = {
        'best_case': {'market_factor': 1.2, 'synergy_factor': 1.3},
        'worst_case': {'market_factor': 0.8, 'synergy_factor': 0.7},
        'realistic': {'market_factor': 1.0, 'synergy_factor': 1.0}
    }

    scenario_results = {}

    for scenario_name, factors in scenarios.items():
        adjusted_roi = roi_metrics['final_roi_percent'] * factors['market_factor'] * factors['synergy_factor']
        scenario_results[scenario_name] = adjusted_roi
        print(f"• {scenario_name}: {adjusted_roi:.1f}% ROI")

    print("✅ What-ifシナリオ分析完了")
    return scenario_results

# 追加分析の実行
sensitivity_results = run_sensitivity_analysis()
scenario_results = run_what_if_scenarios()

print("\n🎊 全ての分析が完了しました！")
print("このコードをGoogle Colabで実行することで、包括的な人的シナジー分析を行うことができます。")
print("必要に応じて、パラメータや分析手法を調整してください。")

# ===============================================================
# STEP 12: インタラクティブグラフをHTMLファイルとして保存
# ===============================================================

import os

# 保存先のフォルダを指定
save_path = "/content/drive/MyDrive/synergy_analysis_charts/"

# フォルダが存在しない場合は作成
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"✅ 保存用フォルダを作成しました: {save_path}")

# all_chartsディクショナリ内の各グラフをHTMLとして保存
for chart_name, chart_figure in all_charts.items():
    file_path = f"{save_path}{chart_name}.html"
    try:
        chart_figure.write_html(file_path)
        print(f"✅ グラフ '{chart_name}' をHTMLファイルとして保存しました: {file_path}")
    except Exception as e:
        print(f"⚠️ '{chart_name}' の保存中にエラーが発生しました: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ ライブラリのインストールと設定が完了しました
✅ Arale Cohenのプロフィールを読み込みました
✅ Yanay Gevaのプロフィールを読み込みました
✅ Yasuyuki Sakaneのプロフィールを読み込みました
✅ プロフィール分析が完了しました
✅ シナジー計算が完了しました
✅ 動的システムモデリングが完了しました
✅ ゲーム理論分析が完了しました
✅ 予測モデリングが完了しました
📊 可視化を生成中...


✅ 全ての可視化が完了しました
✅ レポートが保存されました: /content/drive/MyDrive/synergy_analysis_report_20250620_203819.md
🎉 人的シナジー分析が完了しました！

📊 主要分析結果:
• 平均シナジースコア: 0.485
• 期待ROI: 50866083.0%
• 成功確率: 100.0%

👥 個人別価値貢献度:
• Arale Cohen: 6.966
• Yanay Geva: 6.957
• Yasuyuki Sakane: 7.010

📈 年間成長率予測:
• Arale Cohen: 246.4%
• Yanay Geva: 246.4%
• Yasuyuki Sakane: 246.4%

🎯 戦略的推奨事項:
1. Yasuyuki Sakaneの参画を強く推奨
2. 初期6ヶ月間の文化適応期間を設定
3. 定期的なシナジー効果測定を実施

📋 生成されたアウトプット:
• 8つの可視化チャート
• 包括的な分析レポート
• リスク評価・緩和策
• ROI・財務分析

🚀 分析完了！Googleドライブにレポートが保存されました。

🔍 感度分析を実行中...
✅ 感度分析完了

🔮 What-ifシナリオ分析を実行中...
• best_case: 79351089.5% ROI
• worst_case: 28485006.5% ROI
• realistic: 50866083.0% ROI
✅ What-ifシナリオ分析完了

🎊 全ての分析が完了しました！
このコードをGoogle Colabで実行することで、包括的な人的シナジー分析を行うことができます。
必要に応じて、パラメータや分析手法を調整してください。
✅ グラフ 'skill_radar' をHTMLファイルとして保存しました: /content/drive/MyDrive/synergy_analysis_charts/skill_radar.html
✅ グラフ 'synergy_heatmap' をHTMLファイルとして保存しました: /content/drive/MyDrive/synergy_analysis_charts/synergy_heatmap.html
✅ グラフ 'growth_t